In [5]:
import json

sa_file = "/tmp/2022-06-29_de_serlo.org_events.ndjson"
events = []

for line in open(sa_file, "r").readlines():    
    event = json.loads(line)
    
    if event["datapoint"] == "legacy_serlo_org":
        events.append(event)

events[0:2]

[{'browser_name': 'Samsung Internet',
  'browser_version': '16',
  'datapoint': 'legacy_serlo_org',
  'device_type': 'mobile',
  'document_referrer': None,
  'hostname': 'de.serlo.org',
  'hostname_original': None,
  'is_robot': False,
  'os_name': 'Android',
  'os_version': '11',
  'path': '/abcAlpha -App: Deutsch-Trainer A1Zehn Kapitel mit Wortschatz',
  'user_agent': 'Mozilla/5.0 (Linux; Android 11; SAMSUNG SM-A515F) AppleWebKit/537.36 (KHTML, like Gecko) SamsungBrowser/16.2 Chrome/92.0.0.0 Mobile Safari/537.36'},
 {'browser_name': 'Google Chrome',
  'browser_version': '102',
  'datapoint': 'legacy_serlo_org',
  'device_type': 'desktop',
  'document_referrer': 'http://de.serlo.org/',
  'hostname': 'de.serlo.org',
  'hostname_original': None,
  'is_robot': False,
  'os_name': None,
  'os_version': None,
  'path': '/auth/hydra/login',
  'user_agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}]

In [40]:
import re

known_legacy_paths = {
    "user_management": {
        "login via hydra": "/auth/hydra/login",
        "restore password": "/auth/password/restore",
        "change password": "/auth/password/change",
        "logout": "/auth/hydra/logout",
        "activate account": "/auth/activate",
        "show a role (add user to role)": re.compile("/authorization/role/show/\d+")
    },
    "static pages": {
        "add revision": re.compile("/page/revision/create/.*"),
        "add revision with markdown editor": re.compile("/page/revision/create-old/.*"),
    },
    "taxonomy": {
        "update taxonomy term": re.compile("/taxonomy/term/update/.*"),
        "create new taxonomy term": re.compile("/taxonomy/term/create/.*"),
        "organize taxonomy term children": re.compile("/taxonomy/term/organize/.*"),
        "organize entity children": re.compile("/taxonomy/term/sort/entities/.*")
    },
    "entity": {
        "add revision": re.compile("/entity/repository/add-revision.*")
    },
    "misc": {
        "discussion redirect": re.compile("/discussion/\d+")
    }
}

blacklist = [
    re.compile("/mathe.*"),
    re.compile("/informatik.*"),
    re.compile("/nachhaltigkeit.*"),
    re.compile("/biologie.*"),
    re.compile("/chemie.*"),
    re.compile("/blog.*"),
    re.compile("/\d+"),
    re.compile("/uploads/.*"),
    re.compile("/user/\d+/.*")
]

known_paths = sum((list(k.values()) for k in known_legacy_paths.values()), []) + blacklist
kp_string = [k for k in known_paths if isinstance(k, str)]
kp_re = [k for k in known_paths if not isinstance(k, str)]

def is_known_path(path):
    return path in kp_string or any((r.match(path) for r in kp_re))

paths = [e["path"] for e in events]
paths = set((p for p in paths if not is_known_path(p)))

print("%s path are not catched by the above blacklist / known paths:" % len(paths))
list(paths)

49 path are not catched by the above blacklist / known paths:


['/https://de.serlo.org/mathe/geometrie/analytische-geometrie/abstaende-winkel/abstaende/abstand-zweier-punkte-berechnen',
 '/authorization/forbidden',
 '/auth/logout',
 '/entity/link/order/70975/link',
 '/abcAlpha -App: Deutsch-Trainer A1Zehn Kapitel mit Wortschatz',
 '/',
 '/api/pdf/226928',
 '/api/pdf/62627',
 '/math/26410aufgaben-zur-bestimmung-von-nullstellen',
 '/community -> Hilfe zur Bearbeitung -> Richtlinien für Inhalte',
 '/gemischteaufgabenzugleichungen',
 '/https://de.serlo.org/mathe/geometrie/raeumliche-figuren/wichtige-grundkoerper/quader',
 '/biohttps://studyflix.de/biologie/verdauungsorgane-3106logie/64745/verdauung-des-menschen',
 '/ mathe/ 1555/ Zylinder',
 '/de.serlo.org/user/profile/leogato',
 '/gemischte-Aufgaben-zu-Gleichungen',
 '/auth/password/restore/c5b52472',
 '/community/blog-beiträge-(archiv)/2014-06-29-förderung-hans-sauer-stiftung-',
 '/mitmachen',
 '/https://de.serlo.org/chemie/reaktionen/arten-reaktionen/oxidation-reduktion/redoxreaktion/redox-reaktion